<a href="https://colab.research.google.com/github/conorgibbons147/cyclegan-map/blob/main/train_fixed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
! git clone https://github.com/conorgibbons147/cyclegan-map.git

fatal: destination path 'cyclegan-map' already exists and is not an empty directory.
